## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar10_resnet83_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Resnet83 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarResnetConfig(num_layers = 83, cifar = 10, block_type = 'Bottleneck')
model = model_gen.GetCifarResnet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 1.906651, Test Acc: 39.40 %
2 epoch end, loss: 1.420375, Test Acc: 51.20 %
3 epoch end, loss: 1.092179, Test Acc: 58.17 %
4 epoch end, loss: 0.879076, Test Acc: 67.94 %
5 epoch end, loss: 0.734452, Test Acc: 72.49 %
6 epoch end, loss: 0.639003, Test Acc: 71.48 %
7 epoch end, loss: 0.582432, Test Acc: 79.96 %
8 epoch end, loss: 0.539607, Test Acc: 77.18 %
9 epoch end, loss: 0.500950, Test Acc: 80.22 %
10 epoch end, loss: 0.476307, Test Acc: 80.49 %
11 epoch end, loss: 0.452554, Test Acc: 82.54 %
12 epoch end, loss: 0.427944, Test Acc: 79.19 %
13 epoch end, loss: 0.412460, Test Acc: 79.92 %
14 epoch end, loss: 0.399861, Test Acc: 81.57 %
15 epoch end, loss: 0.385104, Test Acc: 80.85 %
16 epoch end, loss: 0.377587, Test Acc: 85.56 %
17 epoch end, loss: 0.356465, Test Acc: 84.45 %
18 epoch end, loss: 0.350062, Test Acc: 83.29 %
19 epoch end, loss: 0.340201, Test Acc: 83.42 %
20 epoch end, loss: 0.332872, Test Acc: 84.45 %
21 epoch end, loss: 0.327651, Test Acc: 83.56 %
2

171 epoch end, loss: 0.003379, Test Acc: 93.17 %
172 epoch end, loss: 0.003021, Test Acc: 93.24 %
173 epoch end, loss: 0.003158, Test Acc: 93.14 %
174 epoch end, loss: 0.003166, Test Acc: 93.15 %
175 epoch end, loss: 0.003273, Test Acc: 93.13 %
176 epoch end, loss: 0.003044, Test Acc: 93.22 %
177 epoch end, loss: 0.002951, Test Acc: 93.19 %
178 epoch end, loss: 0.003283, Test Acc: 93.21 %
179 epoch end, loss: 0.003076, Test Acc: 93.25 %
180 epoch end, loss: 0.003038, Test Acc: 93.18 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 93.18 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)